In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor

from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df_train = pd.read_csv('/Users/kirillvolosnikov/Desktop/Datasets/tweets_lemm_train.csv')
df_test = pd.read_csv('/Users/kirillvolosnikov/Desktop/Datasets/tweets_lemm_test.csv')

df_train.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


In [4]:
df_test.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [5]:
X_train = df_train.drop('positive', axis = 1)
X_train.head()

,text,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,вот что значит страшилка но блин посмотреть ве...


In [6]:
y_train = df_train['positive']
y_train.head()

0    1
1    1
2    1
3    1
4    1
Name: positive, dtype: int64

In [8]:
X_test = df_test.copy()
X_test.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kirillvolosnikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [15]:
count_tf_idf = TfidfVectorizer(ngram_range=(1, 2))
X_train_tf_idf = count_tf_idf.fit_transform(X_train['lemm_text'])
X_test_tf_idf = count_tf_idf.transform(X_test['lemm_text'])


In [16]:
logistic_model = LogisticRegression()

# Обучаем модель на обучающих данных
logistic_model.fit(X_train_tf_idf, y_train)  # y_train - целевая переменная для обучающей выборки

# Предсказываем значения для валидационных данных
y_pred = logistic_model.predict(X_test_tf_idf)

y_pred

array([1, 1, 0, ..., 0, 1, 1])

In [17]:
lr_results = np.array(list(zip(y_pred)))
lr_results = pd.DataFrame(lr_results, columns=['predictions'])
lr_results.to_csv('predictions', index = False)

In [18]:
import os
os.getcwd()


'/Users/kirillvolosnikov'

In [22]:
df_new = pd.read_csv('predictions')
df_new.head()

,predictions
0,1
1,1
2,0
3,0
4,1


In [23]:
file_path = r'/Users/kirillvolosnikov/Desktop/Datasets/predictions.csv'

# Сохранить DataFrame в файл predictions.csv на рабочем столе
df_new.to_csv(file_path, index=False)